# Segmenting and Clustering Neighborhoods in Toronto

### Basic imports and installations

In [41]:
!pip install lxml
!pip install bs4
!pip install geocoder
import pandas as pd
import lxml
from bs4 import BeautifulSoup
import io
import geocoder

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 98 kB 4.5 MB/s eta 0:00:011
     |████████████████████████████████| 82 kB 1.8 MB/s eta 0:00:011
     |████████████████████████████████| 829 kB 9.8 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491056 sha256=b1e230cfd8a01706f1f710cfffd26b8c79bbfe21a6939b0c660688564c39e446
  Stored in directory: /home/jovyan/.cache/pip/wheels/6e/9c/ed/4499c9865ac1002697793e0ae0

### Page scrapping from wikipedia

In [19]:
#dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
#dfs
!wget -q -O 'neighbor_data.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
df = None  
with open("neighbor_data.html") as fp:
    soup = BeautifulSoup(fp)
    table = soup.find_all('table')[0]
    df = pd.read_html(table.prettify())[0]
    
df

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing Centre
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Dataframe cleaning (we only keep cells with assigned Borough)

df = df[df['Borough'] != 'Not assigned']

df

In [37]:
len(df['Postal Code'].unique())

103

Postal codes are all unique, so no need to merge

### Shape of the dataframe

In [33]:
df.shape

(103, 3)

## Neighborhood coordinates

In [45]:
coord_df = pd.read_csv('https://cocl.us/Geospatial_data')
coord_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


### We integrate the coordinates to our dataframe

In [47]:
df = pd.merge(df,coord_df,on="Postal Code")

In [48]:
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
